In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import calendar

In [2]:
origin = pd.read_csv("/Users/danieldiiorio/Projects/Time-Recorder/Data/export_20200427_162355.csv")
origin.head()

,开始时间,持续时间,事件大类,事件类别,内容类型,事件内容,备注,Tag,状态
0,2020/3/4 0:00,120,未分类,玩手机,未分类,NaN,NaN,NaN,已完成
1,2020/3/4 2:00,450,未分类,睡觉,未分类,NaN,NaN,NaN,已完成
2,2020/3/4 9:30,60,未分类,玩手机,未分类,NaN,NaN,NaN,已完成
3,2020/3/4 10:30,30,未分类,通勤,未分类,NaN,NaN,NaN,已完成
4,2020/3/5 0:00,30,未分类,玩手机,未分类,NaN,NaN,NaN,已完成


In [3]:
data = origin.replace()
temp = data["开始时间"].str.split("/", expand=True)
data["年"] = temp[0]
data["月"] = temp[1].str.rjust(2, "0")
data["临时"] = temp[2]
temp = data["临时"].str.split(" ", expand=True)
data["日"] = temp[0].str.rjust(2, "0")
data["临时"] = temp[1]
temp = data["临时"].str.split(":", expand=True)
data["小时"] = temp[0].str.rjust(2, "0")
data["分"] = temp[1].str.rjust(2, "0")
data["时间"] = data["年"] + data["月"] + data["日"]
data.drop(["开始时间", "事件大类", "内容类型", "备注", "Tag", "状态", "临时"], axis=1, inplace=True)
data.head()

,持续时间,事件类别,事件内容,年,月,日,小时,分,时间
0,120,玩手机,NaN,2020,03,04,00,00,20200304
1,450,睡觉,NaN,2020,03,04,02,00,20200304
2,60,玩手机,NaN,2020,03,04,09,30,20200304
3,30,通勤,NaN,2020,03,04,10,30,20200304
4,30,玩手机,NaN,2020,03,05,00,00,20200305


In [4]:
# 获取数据基础信息
NUM_SAMPLES = data.shape[0]
EVENTS_TYPES = data["事件类别"].unique()
EVENTS = data["事件内容"].unique()
# 整理时间数据
TODAY = datetime.date.today()
# 获取本周第一天
FIRST_DAY_OF_WEEK = TODAY - datetime.timedelta(TODAY.weekday())
# 获取本周最后一天
END_DAY_OF_WEEK = TODAY + datetime.timedelta(6 - TODAY.weekday())
# 获取本月第一天
FIRST_DAY_OF_MONTH = datetime.date(TODAY.year, TODAY.month, 1)
# 获取本月的最后一天
END_DAY_OF_MONTH = datetime.date(TODAY.year,
                                 TODAY.month,
                                 calendar.monthrange(TODAY.year,
                                                     TODAY.month,)[1])
# 获取本年的第一天
FIRST_DAY_OF_YEAR = datetime.date(TODAY.year, 1, 1)
# 获取本年的最后一天
END_DAY_OF_YEAR = datetime.date(TODAY.year + 1, 1, 1) - datetime.timedelta(1)
# 获取上月的最后一天
END_DAY_OF_LAST_MONTH = FIRST_DAY_OF_MONTH - datetime.timedelta(1)
# 获取上月的第一天
FIRST_DAY_OF_LAST_MONTH = datetime.date(END_DAY_OF_LAST_MONTH.year,
                                        END_DAY_OF_LAST_MONTH.month,
                                        1)
# 获取上周的第一天
FIRST_DAY_OF_LAST_WEEK = TODAY - datetime.timedelta(TODAY.weekday() + 7)
# 获取上周的最后一天
END_DAY_OF_LAST_WEEK = TODAY - datetime.timedelta(TODAY.weekday() + 1)

In [5]:
DATES = [TODAY,
         FIRST_DAY_OF_WEEK, END_DAY_OF_WEEK,
         FIRST_DAY_OF_MONTH, END_DAY_OF_MONTH,
         FIRST_DAY_OF_LAST_MONTH, END_DAY_OF_LAST_MONTH,
         FIRST_DAY_OF_YEAR, END_DAY_OF_YEAR]
for index in range(len(DATES)):
    DATES[index] = DATES[index].strftime("%Y%m%d")

In [15]:
# 本周的数据
this_week = data[(data["时间"] >= DATES[5]) & (data["时间"] <= DATES[6])]
this_week

,持续时间,事件类别,事件内容,年,月,日,小时,分,时间
10,540,睡觉,NaN,2020,04,06,00,00,20200406
11,150,玩手机,NaN,2020,04,06,09,00,20200406
12,60,通勤,NaN,2020,04,06,11,30,20200406
13,30,玩手机,NaN,2020,04,06,12,30,20200406
14,180,学习,NaN,2020,04,06,13,00,20200406
...,...,...,...,...,...,...,...,...,...
101,60,玩手机,NaN,2020,04,26,23,00,20200426
102,570,睡觉,NaN,2020,04,27,00,00,20200427
103,90,玩手机,NaN,2020,04,27,09,30,20200427
104,120,通勤,NaN,2020,04,27,11,00,20200427


In [10]:
grouped = data.groupby(["事件类别", "月"])
result = grouped["持续时间"].agg([np.sum, np.mean])
result

sum        mean
事件类别 月                   
学习   03    30   30.000000
     04  1860  132.857143
工作   04   180   60.000000
玩手机  03   270   54.000000
     04  2490   80.322581
玩游戏  04  2040  185.454545
看电影  04   480  120.000000
睡觉   03   960  480.000000
     04  7020  585.000000
聚会   04   660  660.000000
通勤   03    60   30.000000
     04  1320   66.000000